In [1]:
import cv2
from cnn import CNN
import tensorflow as tf
cnn = CNN()

In [2]:
data_list = [
    {"folder":"image_t_face","label":"0"},
    {"folder":"image_f_face","label":"1"}
]
    
    
test_list = [
    {"folder":"img_t_face_test", "label":"0"}
    #{"folder":"img_f_face_test", "label":"1"}
]

train_image, train_label, _ = cnn.read_image_cv(data_list)
test_image , test_label , test_image_list = cnn.read_image_cv(test_list)  

In [3]:
len(train_image)
import os
log_dir = os.getcwd() + cnn.LOG_DIR
log_dir

'/home/tamotoyoshifumi/GIT/is-this-obama/domain/var/log'

In [4]:
with tf.Graph().as_default():
        
    images_placeholder = tf.placeholder("float", shape=(None, cnn.IMAGE_PIXELS))
    labels_placeholder = tf.placeholder("float", shape=(None, cnn.NUM_CLASSES))
    keep_prob = tf.placeholder("float")

    
    softmax = cnn.inference(images_placeholder, keep_prob)
    loss_value = cnn.loss(softmax, labels_placeholder)
    train_op = cnn.training(loss_value, cnn.LEARNING_RATE)
    acc = cnn.accuracy(softmax, labels_placeholder)
    
    saver = tf.train.Saver()

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
            
    summary_op = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
        
        
    for step in range(cnn.MAX_STEPS):
        for i in range(len(train_image)/cnn.BATCH_SIZE):
            batch = cnn.BATCH_SIZE*i
            sess.run(train_op, feed_dict={
                    images_placeholder: train_image[batch:batch+cnn.BATCH_SIZE],
                    labels_placeholder: train_label[batch:batch+cnn.BATCH_SIZE],
                    keep_prob: 0.5})

        #train_accuracy = sess.run(acc, feed_dict={
        #        images_placeholder: train_image,
        #        labels_placeholder: train_label,
        #        keep_prob: 1.0})
        #print "step %d, training accuracy %g"%(step, train_accuracy)

        p = sess.run(softmax,  feed_dict={
            images_placeholder: train_image,
            keep_prob: 1.0})
        print "step %d, p %s"%(step, str(p))
        
        train_loss = sess.run(loss_value,  feed_dict={
            images_placeholder: train_image,
            labels_placeholder: train_label,
            keep_prob: 1.0})
        print "step %d, xentropy %s"%(step, str(train_loss))
            
        if step % 10 == 0 and step > 0 and train_loss < 5.0:
            cnn.display_test_prob(sess, softmax, test_image, "obama", images_placeholder,keep_prob)

        summary_str = sess.run(summary_op, feed_dict={
            images_placeholder: train_image,
            labels_placeholder: train_label,
            keep_prob: 1.0})
        summary_writer.add_summary(summary_str, step)

  
    print "test accuracy %g"%sess.run(acc, feed_dict={
        images_placeholder: test_image,
        labels_placeholder: test_label,
        keep_prob: 1.0})
    

    save_path = saver.save(sess, "model.ckpt")

step 0, p [[  4.94847208e-23   1.00000000e+00]
 [  2.94432627e-22   1.00000000e+00]
 [  1.03552660e-19   1.00000000e+00]
 ..., 
 [  1.60717566e-25   1.00000000e+00]
 [  1.06864071e-22   1.00000000e+00]
 [  3.41509271e-19   1.00000000e+00]]
step 0, xentropy 12135.6
step 1, p [[  3.35110999e-05   9.99966502e-01]
 [  7.42242992e-05   9.99925733e-01]
 [  1.83589073e-05   9.99981642e-01]
 ..., 
 [  4.69808674e-06   9.99995351e-01]
 [  3.34533411e-06   9.99996662e-01]
 [  7.77168025e-05   9.99922276e-01]]
step 1, xentropy 2434.23
step 2, p [[  4.19152249e-03   9.95808482e-01]
 [  3.39221978e-03   9.96607780e-01]
 [  2.63366476e-03   9.97366369e-01]
 ..., 
 [  6.25250745e-04   9.99374807e-01]
 [  6.21410436e-05   9.99937892e-01]
 [  1.13838236e-03   9.98861670e-01]]
step 2, xentropy 1248.75
step 3, p [[  4.96807508e-02   9.50319290e-01]
 [  5.64293452e-02   9.43570614e-01]
 [  1.91883072e-02   9.80811715e-01]
 ..., 
 [  1.98767986e-02   9.80123222e-01]
 [  9.07204289e-04   9.99092817e-01]
 [ 

In [7]:
import os
import getface as gf
cwd = os.getcwd()
image_path = cwd + "/image_t"
files = os.listdir(image_path)
len(files)

313

In [10]:
for f in files:
    dist_path = cwd+"/image_t_face"
    image_name = f
    r = gf.cutout_face(image_path, image_name, dist_path)
    print r

{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'error', 'code': 1, 'http_status': 400}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'error', 'code': 1, 'http_status': 400}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 2}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type'

In [9]:
dist_path = cwd+"/image_t_face"
len(os.listdir(dist_path))

255

In [12]:
import os
import getface as gf
cwd = os.getcwd()
image_path = cwd + "/image"
files = os.listdir(image_path)
len(files)

1001

In [13]:
for f in files:
    dist_path = cwd+"/image_f_face"
    image_name = f
    r = gf.cutout_face(image_path, image_name, dist_path)
    print r

{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 4}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'error', 'code': 1, 'http_status': 400}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'error', 'code': 1, 'http_status': 400}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type': 'detail'}
{'status': 'success', 'detail': {'number': 1}, 'data_type'